<a href="https://colab.research.google.com/github/Javedz678/resume-screener/blob/main/ETL_Resume_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Extract text from resume</h1>

In [ ]:
!pip install sentence_transformers
!pip install pyresparser
!python -m spacy download en_core_web_sm
!pip install spacy
!pip install gingerit

In [ ]:
import string
import re
import nltk
import statistics
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from tabulate import tabulate
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import os
from tqdm import tqdm
import en_core_web_sm
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
os.chdir("/content/resume/")
from pyresparser import ResumeParser
from gingerit.gingerit import GingerIt
import json

In [ ]:
from tqdm.notebook import tqdm_notebook

In [ ]:
#Parsing
def parse(path2file):
  return ResumeParser(path2file).get_extracted_data()

In [ ]:
def start():
  resume_list = os.listdir()
  for resume in resume_list:
    print("Parsing.....")
    parse(resume)

In [ ]:
#Global arrays
score_array=[]
master_records= []
top_institutes = ["IIT","NIT","IIIT","BITS"]
equvalent_courses = ["B.E","MSC","MCA","B.Tech","Bachelors","Information science","AI","ML"]
top_banks = ["JP Morgan","Goldman sachs","Axis bank","HSBC","VISA","PayPal","PWC","HDFC Bank"]
prefer = False

Data cleaning

In [ ]:
def start(actual,expected):
  removeStopwords(actual,expected)

In [ ]:
def checkTopInstitutes(education,requiredEducation):
  for institute in top_institutes:
    if institute in education:
      education = requiredEducation
      break
    else:
      education.join(',')

In [ ]:
def checkEquvalentCourses(education,requiredEducation):
  for course in equvalent_courses:
    if course in education:
      education = requiredEducation
    else:
      education.join(',')

In [ ]:
def preferExperience(candidateExperience):
  candidateExperience = candidateExperience.split()
  for bank in top_banks:
    if bank in candidateExperience:
      return True

In [ ]:
def removeStopwords(actual,expected):
  stop_words = set(stopwords.words('english'))
  candidateExperienceTokens = word_tokenize(actual)
  requiredExperienceTokens = word_tokenize(expected)
  cleanCandidateExperience = [word for word in candidateExperienceTokens if not word in stopwords.words()]
  cleanrequiredExperience = [word for word in requiredExperienceTokens if not word in stopwords.words()]
  lemmatisation(cleanCandidateExperience,cleanrequiredExperience)

In [ ]:
def lemmatisation(actual,expected):
  CandidateExperience = ' '.join(map(str, actual))
  RequiredExperience = ' '.join(map(str, expected))
  lemmatizer = WordNetLemmatizer()
  lemCandidateExperience =lemmatizer.lemmatize(CandidateExperience)
  lemRequiredExperience = lemmatizer.lemmatize(RequiredExperience)
  removePunctuation(lemCandidateExperience,lemRequiredExperience)

In [ ]:
def removePunctuation(actual,expected):
  noPunCandidateExperience = ""
  noPunRequiredExperience = ""
  for character in actual:
      if character.isalnum():
          noPunCandidateExperience += character
      else:
        noPunCandidateExperience += " "
        
  for character in expected:
      if character.isalnum():
          noPunRequiredExperience += character
      else:
        noPunRequiredExperience += " "
  model(noPunCandidateExperience,noPunRequiredExperience)

In [ ]:
def drawPieChart():
  y=[]
  for i in range(len(score_array)):
    if(i==2):
      y.append(score_array[i]*1.5)
    else:
      y.append(score_array[i])
  mylabels = ["Skills", "Education", "Experience"]
  explode = (0, 0, 0.1 ) 
  plt.pie(y, labels = mylabels, explode=explode, shadow=True)
  plt.title("Distribution of attributes of the Candidate.")
  plt.show() 

In [ ]:
#Process prequesit requirements 
def processPrequesits(requireTopTierEducation,preferBankExperience):
  if(requireTopTierEducation):
    requiredEducation = "BE in computer science from" + ' '.join([str(elem) for elem in top_institutes])
    checkTopInstitutes(education,requiredEducation)
  else:
    requiredEducation = "BE in computer science "
    checkEquvalentCourses(education,requiredEducation)
  if(preferBankExperience):
    hasPreferance = preferExperience(candidateExperience)
    if(hasPreferance):
      prefer = True
  return requiredEducation

In [ ]:
def initJob(requireTopTierEducation,preferBankExperience,requirementCorpus,prefer = False):
  #requiredEducation = processPrequesits(requireTopTierEducation,preferBankExperience)
  resume_list = os.listdir()
  for resume in tqdm_notebook(resume_list):
    obj = parse(resume)
    data = obj.get('experience')
    candidateCorpus = "".join(data)
    sentenceTranformerModel(candidateCorpus.lower(),requirementCorpus.lower())
    analytics(resume,prefer)
    
  #drawPieChart()
  #analytics(prefer)

In [ ]:
def analytics(name,prefer = False):
  avg = score_array[0]
  score_array.append(name)
  if(avg>min_criteria):
    score_array.append("PASSED")
  elif(avg>70 and avg<min_criteria):
    score_array.append("Manual review needed")
  else:
    score_array.append("FAILED")

  if(prefer):
    score_array.append("Has banking experience.")
  print(score_array)
  score_array.clear()

In [ ]:
def showTable():
  print(master_records)
  #print(tabulate(master_records, headers=["Resume score","Automatic screening","Insights"]))

<h3>Modelling</h3>

In [ ]:
def sentenceTranformerModel(actual,expected):
  modelName = "bert-base-nli-mean-tokens"
  model = SentenceTransformer(modelName)
  vectorA = model.encode([actual])
  vectorB = model.encode([expected])
  similarity_index = cosine_similarity(vectorA, vectorB)
  final_score = similarity_index[0][0]*100
  score_array.append(int(final_score))

<h3>Analysis</h3>


Make changes below to describe what is needed.

## **Runnable**

***Organisation - Taken from front end***

In [ ]:
requireTopTierEducation = False
preferBankExperience = False
min_criteria = 80
requiredEducation = "Bachelors of engineering "
requiredSkills = " Java, REACT JS, Spring, Hibernate, CI/ CD, Docker, Kubernetes, Public Cloud (Azure), SQL, Web technologies."
requiredExperience = " Deep expertise and hands on experience in Core java. · Hands-on experience with at least 2 to 12 years in Spring and Hibernate technologies. · Hands-on experience with at least 2 to 12 years in web GUI development using HTML, CSS, ReactJS, Bootstrap · Clear understanding of Software Development life cycle and process Oriented. Would be great if worked on Agile or Continuous Delivery (CD) based projects before. · Good fundamentals in Core Programming, Object Oriented design & Design patterns · Should have good hands -on experience on the fundamentals of how modern web works with SPA, Restful APIs and OAuth/OpenID Connect protocols · Experience in creating & executing unit tests using test driven approach with JUnit or any other equivalent."
requirementCorpus = requiredEducation + requiredSkills + requiredExperience

***Candidates - Taken from Resume***

In [ ]:
#Start Job
initJob(requireTopTierEducation,preferBankExperience,requirementCorpus,prefer)

  0%|          | 0/6 [00:00<?, ?it/s]

[72, 'Galla Renuka.pdf', 'Manual review needed']
[67, 'RakeshM.pdf', 'FAILED']
[68, 'SivaKumar.pdf', 'FAILED']
[63, 'Yesuraju.pdf', 'FAILED']
[70, 'Anamala SandeepReddy.pdf', 'FAILED']
[58, 'VaraPrasad.pdf', 'FAILED']
